In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/My \Drive/Sem \8 \files/
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Sem 8 files
 data				  __pycache__
 euclidean_distances.json	  Shortest_path_lengths.pkl
'Fri Feb 14 09_06_16 2020.json'   utils.py


In [2]:
!apt-get install -y libspatialindex-c4v5
!pip3.6 install osmnx

import osmnx as ox
import networkx as nx
import pandas as pd
import json
import random
import statistics

from rtree import index
from utils import *

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-c4v5 is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [3]:
place_name = "Singapore, Central, Singapore"
G = ox.save_load.load_graphml(filename="Singapore_drive_processed.graphml")

# Stores nodes and edges mapped with speed bands along with other attributes
type(G)

networkx.classes.multidigraph.MultiDiGraph

In [0]:
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
nodes = pd.read_pickle('data/nodes_drive.pkl')

In [5]:
f = open("Fri Feb 14 09_06_16 2020.json", "r").read()
x = json.loads(f)

speed_bands = pd.DataFrame.from_dict(x, orient='columns')
process_speed_band(speed_bands)

/content/drive/My Drive/Sem 8 files/utils.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MaximumSpeed'][i] = '80'


In [0]:
calculate_congestion(G, edges, speed_bands)

### Calculate Variance

In [0]:
# key: node label ,value: location
idx = index.Index()

xx = nodes['x'].astype(float)
yy = nodes['y'].astype(float)
node_id = nodes['osmid']
node_type = nodes['highway']

# nan, 'crossing', 'turning_circle', 'traffic_signals',
# 'motorway_junction', 'turning_loop', 'elevator', 'bus_stop',
# 'mini_roundabout', 'stop', 'speed_camera'

for node, x, y, kind in zip(node_id, xx, yy, node_type):
    if kind != 'nan' and kind != 'stop' and kind != 'speed_camera':
        idx.insert(int(node), (x,y,x,y))

In [0]:
def find_nearest_nodes(source, dist, idx):
    # Create a bounding box around source of min distance in all directions
    (north, south, east, west) = ox.bbox_from_point(point=source, distance=dist)
    
    candidate_nodes = list(idx.intersection((west, south, east, north)))    
    return candidate_nodes

In [0]:
def get_length_dict(sources, destinations, method='euclidean'):
    distances = dict()
    
    if method == 'euclidean':
        for source in sources:        
            for dest in destinations:
                ed = haversine_distance(nodes, source, dest)
                distances[source,dest] = ed/13.8889
    else:
        for source in sources:        
            for dest in destinations:
                try:
                    # dijsktra distance in sec
                    d = nx.shortest_path_length(G, source, dest, weight='BPR', method='dijkstra')
                except nx.NetworkXUnfeasible:
                    d = float('inf')
                distances[(source,dest)] = d
       
    # sort the dictionary in ascending order using distances
    x = sorted(distances.items(), key = lambda kv:(kv[1], kv[0]))
    return x    

In [0]:
def lets_do_this():
  # get 20 random sources
  sources = []
  for i in range(20):
      r = random.randint(0,23210)
      s = nodes['osmid'].values[r]
      sources.append(s)
      
  # get 20 correspondig random targets
  targets = []
  for i in range(20):
      r = random.randint(0,23210)
      t = nodes['osmid'].values[r]
      targets.append(t)

  deviations = []
  for s,t in zip(sources,targets):
      orig_point = nodes[nodes['osmid'] == s].geometry.values[0]
      target_point = nodes[nodes['osmid'] == t].geometry.values[0]
      orig_xy = (orig_point.y, orig_point.x)
      target_xy = (target_point.y, target_point.x)
      
      # Max distance for walking is 720m (10 min * 1.2 m/s)
      # Find all nodes within 720 m from candidate sources and candidate destinations
      candidate_dest = find_nearest_nodes(target_xy, 720, idx)
      candidate_source = find_nearest_nodes(orig_xy, 720, idx)
      
      real_lengths = dict(get_length_dict(candidate_source, candidate_dest, method='dijkstra'))
      euclidean = get_length_dict(candidate_source, candidate_dest)
      
      # value1: hybrid, value2: exhaustive
      value1 = hybrid_search_length(G, euclidean, real_lengths)
      key = next(iter(real_lengths))
      
      value2 = real_lengths[key]
      stddev = statistics.stdev([value1, value2])

      deviations.append(stddev)
      print(s, t, value1, value2, stddev)
      
  # take mean of deviations
  print(statistics.mean(deviations))

In [11]:
lets_do_this()

7108062072 4606359563 3595.5602510273407 3595.5602510273407 0.0
246580153 392296057 1539.3407839939646 1539.3407839939646 0.0
4726412914 3614360109 176.82492250567662 176.82492250567662 0.0
3867177044 5683994748 1161.3222538967314 1161.3222538967314 0.0
244322101 247432260 2242.6449007147826 2242.6449007147826 0.0


KeyboardInterrupt: ignored

## Initially when dividing euclidean distances with 5.28 m/s, was getting mean deviation = 190.1719s ~ 3.1 min